# DeepRank-Core basic Protein-Protein Interface data processing

## Introduction

<img style="margin-left: 1.5rem" align="right" src="images/data_generation_ppi.png" width="400">

This tutorial will demonstrate the use of DeepRank-Core for generating Protein-Protein Interfaces (PPIs) starting from [PBD files](https://en.wikipedia.org/wiki/Protein_Data_Bank_(file_format)) representing protein-protein complexes. 

In this data processing phase, for each protein-protein complex an interface is selected according to the user's radius of interest, and it is mapped to a graph, in which nodes represent either residues or atoms, and edges the interactions between them. Nodes and edges can have several different features (i.e. node and/or edge features), which are generated and added during the processing phase as well. Optionally, the graphs can be mapped to volumetric grids. The mapped data are finally saved into [HDF5 files](https://en.wikipedia.org/wiki/Hierarchical_Data_Format), and can be used for later models' training.

### Use case


<img style="margin-right: 1.5rem" align="left" src="images/pmhc_pdb_example.png" width="200"/>

The example dataset that we provide contains PDB files representing [Major Histocompatibility Complex (MHC) protein](https://en.wikipedia.org/wiki/Major_histocompatibility_complex) + peptide (pMHC) complexes, which play a key role in T-cell immunity. We are interested in predicting the Binding Affinity (BA) of the complexes, which can be used to determine the most suitable mutated tumor peptides as vaccine candidates.

PDB models used in this tutorial have been generated with [PANDORA](https://github.com/X-lab-3D/PANDORA), an anchor restrained modeling pipeline for generating peptide-MHC structures. An example of a pMHC structure rendered using [ProteinViewer](https://marketplace.visualstudio.com/items?itemName=ArianJamasb.protein-viewer).

The MHC protein is displayed in green, while the peptide is in orange.

## Setup

### Data

The example data used in this tutorial are available on [tdb, zenodo?](). To download the data, please [visit the dataset page for this workshop and click the “Download all” button. Unzip the downloaded file, and] save the contents as a folder called data in the same directory of this notebook. (The name data and the folder location are optional but recommended, as this are the name and the location we will use to refer to the folder throughout the tutorial.)

The dataset contains only 100 data points, which are obviously not enough to develop an impactful predictive model, and the scope of its use is indeed only demonstrative and informative for the users.

### Software

1. Follow the [updated instructions](https://github.com/DeepRank/deeprank-core#installation) in the README.md on the main branch for successfully installing `deeprankcore` package.
2. Test the environment in which` deeprankcore` has been (successfully) installed. First clone [deeprank-core repository](https://github.com/DeepRank/deeprank-core), navigate into it, and after having activated the environment and installed [pytest](https://anaconda.org/anaconda/pytest), run `pytest tests`. All tests should pass. We recommend installing `deeprankcore` and all its dependencies into a [conda](https://docs.conda.io/en/latest/) environment. 

## Utilities

### Libraries

Let's import the libraries needed for this tutorial:

In [33]:
import os
import pandas as pd
import glob
import h5py
from deeprankcore.query import QueryCollection
from deeprankcore.query import ProteinProteinInterfaceResidueQuery, ProteinProteinInterfaceAtomicQuery
from deeprankcore.utils.grid import GridSettings, MapMethod
from deeprankcore.dataset import GraphDataset

### Paths and raw files

Let's define the paths for reading raw data and saving the processed ones.

`get_pdb_files_and_target_data` is an helper function used to retrieve the raw pdb files names in a list and the binding affinity target values from a CSV containing the IDs of the PDB models as well.

In [13]:
data_path = os.path.join("data", "ppi")
processed_data_path = os.path.join(data_path, "processed")

def get_pdb_files_and_target_data(data_path):
	csv_data = pd.read_csv(os.path.join(data_path, "BA_values.csv"))
	pdb_files = glob.glob(os.path.join(data_path, "pdb", '*.pdb'))
	pdb_files.sort()
	pdb_ids_csv = [pdb_file.split('/')[-1].split('.')[0] for pdb_file in pdb_files]
	csv_data_indexed = csv_data.set_index('ID')
	csv_data_indexed = csv_data_indexed.loc[pdb_ids_csv]
	bas = csv_data_indexed.measurement_value.values.tolist()
	return pdb_files, bas

pdb_files, bas = get_pdb_files_and_target_data(data_path)

## `QueryCollection` and `Query` objects

For each protein-protein complex, so for each data point, a query can be created and added to the `QueryCollection` object, to be processed later on. Different types of queries exist, based on the molecular resolution needed:

- In a `ProteinProteinInterfaceResidueQuery` each node represents one amino acid residue.
- In a `ProteinProteinInterfaceAtomicQuery` each node represents one atom within the amino acid residues.

A query takes as inputs:

- A `.pdb` file, representing the protein-protein structural complex.
- The ids of the two chains composing the complex. In our use case, "M" indicates the protein's chain and "P" the peptide's chain.
- The correspondent [Position-Specific Scoring Matrices (PSSMs)](https://en.wikipedia.org/wiki/Position_weight_matrix), in the form of .pssm files. The PSSM is optional and we are not going to use it in this basic tutorial.
- The distance cutoff, which represents the maximum distance in Ångström between two interacting residues of the two proteins.
- The target values associated with the query. For each query/data point, in the use case demonstrated in this tutorial will add two targets: "BA" and "binary". The first represents the continuous Binding Affinity value of the complex, while the second represents its binary representation, being 0 (BA > 500 nM) a not-binding complex and 1 (BA <= 500 nM) binding one.

## Residue-level PPI: `ProteinProteinInterfaceResidueQuery`

In [3]:
queries = QueryCollection()

interface_distance_cutoff = 15  # max distance in Å between two interacting residues/atoms of two proteins

print(f'Adding {len(pdb_files)} queries to the query collection ...')
count = 0
for i in range(len(pdb_files)):
	queries.add(
		ProteinProteinInterfaceResidueQuery(
			pdb_path = pdb_files[i], 
			chain_id1 = "M",
			chain_id2 = "P",
			distance_cutoff = interface_distance_cutoff,
			targets = {
				'binary': int(float(bas[i]) <= 500), # binary target value
				'BA': bas[i], # continuous target value
				}))
	count +=1
	if count % 20 == 0:
		print(f'{count} queries added to the collection.')

print(f'Queries ready to be processed.\n')

Adding 100 queries to the query collection ...
20 queries added to the collection.
40 queries added to the collection.
60 queries added to the collection.
80 queries added to the collection.
100 queries added to the collection.
Queries ready to be processed.



### Notes on `process()` method

When queries are ready to be processed, we can process them. But first, be sure to set `process()` method parameters according to your needs.

- `feature_modules` parameter can be used for indicating features modules to be used to generate the features. By default, only the basic features contained in `deeprankcore.features.components` and `deeprankcore.features.contact` are generated. A complete and detailed list of the pre-implemented features per module can be found [here](https://deeprankcore.readthedocs.io/en/latest/features.html).
- If you want to include grids in the HDF5 files, which represent the mapping of the graphs to a volumetric box, you need to define `grid_settings` and `grid_map_method` parameters and to pass them to the `process()` method, as shown in the example below. If they are `None`, only graphs are saved.
- `cpu_count` parameter can be used to specify how many processes to be run simultaneously, and will coincide with the number of HDF5 files generated. By default it takes all available CPU cores.


In [4]:
grid_settings = GridSettings( # None if you don't want grids
	# the number of points on the x, y, z edges of the cube
	points_counts = [35, 30, 30],
	# x, y, z sizes of the box in Å
	sizes = [1.0, 1.0, 1.0])
grid_map_method = MapMethod.GAUSSIAN # None if you don't want grids

queries.process(
	os.path.join(processed_data_path, "residue", "proc"),
	combine_output = False,
	grid_settings = grid_settings,
	grid_map_method = grid_map_method)

print(f'The queries processing is done. The generated hdf5 files are in {processed_data_path}.')

The queries processing is done. The generated hdf5 files are in data/ppi/processed_residue.


### Exploring data

As representative example, the following is the HDF5 structure generated by the previous code for `BA-100600.pdb`, so for one single graph, which represents one PPI, for the graph + grid case:

```bash
└── residue-ppi:M-P:BA-100600
    |
    ├── edge_features
    │   ├── _index
    │   ├── _name
    │   ├── covalent
    │   ├── distance
    │   ├── electrostatic
    │   ├── same_chain
    │   └── vanderwaals
    |
    ├── node_features
    │   ├── _chain_id
    │   ├── _name
    │   ├── _position
    │   ├── hb_acceptors
    │   ├── hb_donors
    │   ├── polarity
    │   ├── res_charge
    │   ├── res_mass
    |   ├── res_pI
    |   ├── res_size
    |   └── res_type
    |
    ├── grid_points
    │   ├── center
    │   ├── x
    │   ├── y
    │   └── z
    |
    ├── mapped_features
    │   ├── _position_000
    │   ├── _position_001
    │   ├── _position_002
    │   ├── covalent
    │   ├── distance
    │   ├── electrostatic
    │   ├── polarity_000
    │   ├── polarity_001
    │   ├── polarity_002
    │   ├── polarity_003
    |   ├── ...
    |   └── vanderwaals
    |
    └── target_values
    │   ├── BA
        └── binary
```

`edge_features`, `node_features`, `mapped_features` are [HDF5 Groups](https://docs.h5py.org/en/stable/high/group.html) which contain [HDF5 Datasets](https://docs.h5py.org/en/stable/high/dataset.html) (e.g., `_index`, `electrostatic`, etc.), which in turn contains features values in the form of arrays. `edge_features` and `node_features` refer specificly to the graph representation, while `grid_points` and `mapped_features` refer to the grid mapped from the graph. Each data point generated by deeprankcore has the above structure, with the features and the target changing according to the user's settings. Features starting with `_` are used during the graphs and grids computations, but they are not supposed to be used for training models. 

It is always a good practice to first explore the data, and then make decision about splitting them in training, test and validation sets. There are different possible ways for doing it.



#### Pandas dataframe

The edge and node features just generated can be explored by instantiating the `GraphDataset` object, and then using `hdf5_to_pandas` method which converts node and edge features into a [Pandas](https://pandas.pydata.org/) dataframe. Each row represents a ppi in the form of a graph.  

In [31]:
processed_data = glob.glob(os.path.join(processed_data_path, "residue", "*.hdf5"))
dataset = GraphDataset(processed_data)
df = dataset.hdf5_to_pandas()
df.head()

   ['data/ppi/processed/residue/proc-28835.hdf5', 'data/ppi/processed/residue/proc-28842.hdf5', 'data/ppi/processed/residue/proc-28839.hdf5', 'data/ppi/processed/residue/proc-28838.hdf5', 'data/ppi/processed/residue/proc-28843.hdf5', 'data/ppi/processed/residue/proc-28834.hdf5', 'data/ppi/processed/residue/proc-28837.hdf5', 'data/ppi/processed/residue/proc-28840.hdf5', 'data/ppi/processed/residue/proc-28841.hdf5', 'data/ppi/processed/residue/proc-28836.hdf5'] dataset                 : 100%|██████████| 10/10 [00:00<00:00, 1212.16it/s, entry_name=proc-28836.hdf5]


,id,hb_acceptors,hb_donors,polarity_0,polarity_1,polarity_2,polarity_3,res_charge,res_mass,res_pI,...,res_type_15,res_type_16,res_type_17,res_type_18,res_type_19,covalent,distance,electrostatic,same_chain,vanderwaals
0,residue-ppi:M-P:BA-102611,"[1, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, ...","[1, 0, 1, 0, 0, 0, 0, 3, 2, 1, 0, 0, 0, 0, 3, ...","[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[137.1, 99.1, 101.1, 113.2, 99.1, 113.2, 57.1,...","[7.59, 5.96, 5.6, 6.02, 5.96, 5.98, 5.97, 9.74...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.0036500483676365, 5.84374580213753, 7.58259...","[2.8903641747696636, 6.110196978228215, 3.0734...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[-0.11455826342593298, -0.4672081421955052, -0..."
1,residue-ppi:M-P:BA-102669,"[1, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 4, ...","[1, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 3, 0, ...","[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[137.1, 99.1, 101.1, 113.2, 99.1, 113.2, 57.1,...","[7.59, 5.96, 5.6, 6.02, 5.96, 5.98, 5.97, 5.65...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.025896455257506, 5.9700658287827935, 7.5189...","[-1.1110179510997469, 6.071846611134136, 3.100...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[-0.07806929175638432, -0.44372330471150295, -..."
2,residue-ppi:M-P:BA-102719,"[1, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 4, ...","[1, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 3, 0, ...","[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[137.1, 99.1, 101.1, 113.2, 99.1, 113.2, 57.1,...","[7.59, 5.96, 5.6, 6.02, 5.96, 5.98, 5.97, 5.65...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.0069163688458564, 6.00083877470475, 7.46990...","[3.0206362925557197, 1.4149505177303556, 3.142...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[-0.08981257450837221, -0.3997677749688828, -0..."
3,residue-ppi:M-P:BA-114468,"[1, 0, 2, 0, 0, 0, 4, 0, 0, 2, 0, 0, 0, 0, 4, ...","[1, 0, 1, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, ...","[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 1.0,...","[137.1, 99.1, 101.1, 113.2, 99.1, 113.2, 129.1...","[7.59, 5.96, 5.6, 6.02, 5.96, 5.98, 3.22, 5.97...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0

We can also generate histograms for looking at the features distributions:

In [32]:
dataset.save_hist(
    features = ["res_mass", "distance", "electrostatic"],
    fname = os.path.join(processed_data_path, "residue", "_".join(["res_mass", "distance", "electrostatic"])))

#### Other tools

- [HDFView](https://www.hdfgroup.org/downloads/hdfview/), a visual tool written in Java for browsing and editing HDF5 files.
  As representative example, the following is the structure for `BA-100600.pdb` seen from HDF5View:
  
  <img style="margin-bottom: 1.5rem" align="centrum" src="images/hdfview.png" width="200">

  Using this tool you can inspect the values of the features visually, for each data point. 

- Python packages such as [h5py](https://docs.h5py.org/en/stable/index.html). Examples:

In [42]:
with h5py.File(processed_data[0], "r") as hdf5:
    # List of all graphs in hdf5, each graph representing a ppi
    ids = list(hdf5.keys())
    print(f'IDs of PPIs in {processed_data[0]}: {ids}')
    node_features = list(hdf5[ids[0]]["node_features"]) 
    print(f'Node features: {node_features}')
    edge_features = list(hdf5[ids[0]]["edge_features"])
    print(f'Edge features: {edge_features}')
    targets = list(hdf5[ids[0]]["target_values"])
    print(f'Targets features: {targets}')
    # Polarity feature for ids[0], numpy.ndarray
    node_feat_polarity = hdf5[ids[0]]["node_features"]["polarity"][:]
    print(f'Polarity feature shape: {node_feat_polarity.shape}')
    # Electrostatic feature for ids[0], numpy.ndarray
    edge_feat_electrostatic = hdf5[ids[0]]["edge_features"]["electrostatic"][:]
    print(f'Electrostatic feature shape: {edge_feat_electrostatic.shape}')

IDs of PPIs in data/ppi/processed/residue/proc-28835.hdf5: ['residue-ppi:M-P:BA-102611', 'residue-ppi:M-P:BA-102669', 'residue-ppi:M-P:BA-102719', 'residue-ppi:M-P:BA-114468', 'residue-ppi:M-P:BA-115138', 'residue-ppi:M-P:BA-115586', 'residue-ppi:M-P:BA-128164', 'residue-ppi:M-P:BA-128373', 'residue-ppi:M-P:BA-129219', 'residue-ppi:M-P:BA-153205', 'residue-ppi:M-P:BA-153504', 'residue-ppi:M-P:BA-153607']
Node features: ['_chain_id', '_name', '_position', 'hb_acceptors', 'hb_donors', 'polarity', 'res_charge', 'res_mass', 'res_pI', 'res_size', 'res_type']
Edge features: ['_index', '_name', 'covalent', 'distance', 'electrostatic', 'same_chain', 'vanderwaals']
Targets features: ['BA', 'binary']
Polarity feature shape: (166, 4)
Electrostatic feature shape: (5700,)


## Atomic-level PPI: `ProteinProteinInterfaceAtomicQuery`

Now we will generate data at atomic resolution, very similarly to what we have just done for residue-level. 

In [9]:
queries = QueryCollection()

interface_distance_cutoff = 5.5  # max distance in Å between two interacting residues/atoms of two proteins

print(f'Adding {len(pdb_files)} queries to the query collection ...')
count = 0
for i in range(len(pdb_files)):
	queries.add(
		ProteinProteinInterfaceAtomicQuery(
			pdb_path = pdb_files[i], 
			chain_id1 = "M",
			chain_id2 = "P",
			distance_cutoff = interface_distance_cutoff,
			targets = {
				'binary': int(float(bas[i]) <= 500), # binary target value
				'BA': bas[i], # continuous target value
				}))
	count +=1
	if count % 20 == 0:
		print(f'{count} queries added to the collection.')

print(f'Queries ready to be processed.\n')

Adding 100 queries to the query collection ...
20 queries added to the collection.
40 queries added to the collection.
60 queries added to the collection.
80 queries added to the collection.
100 queries added to the collection.
Queries ready to be processed.



In [14]:
grid_settings = GridSettings( # None if you don't want grids
	# the number of points on the x, y, z edges of the cube
	points_counts = [35, 30, 30],
	# x, y, z sizes of the box in Å
	sizes = [1.0, 1.0, 1.0])
grid_map_method = MapMethod.GAUSSIAN # None if you don't want grids

queries.process(
	os.path.join(processed_data_path, "atomic", "proc"),
	combine_output = False,
	grid_settings = grid_settings,
	grid_map_method = grid_map_method)

print(f'The queries processing is done. The generated hdf5 files are in {processed_data_path}.')

The queries processing is done. The generated hdf5 files are in data/ppi/processed.


Again, we can give a look at the data using `hdf5_to_pandas` function.

In [28]:
processed_data = glob.glob(os.path.join(processed_data_path, "atomic", "*.hdf5"))
dataset = GraphDataset(processed_data)
df = dataset.hdf5_to_pandas()
df.head()

   ['data/ppi/processed/atomic/proc-29166.hdf5', 'data/ppi/processed/atomic/proc-29170.hdf5', 'data/ppi/processed/atomic/proc-29171.hdf5', 'data/ppi/processed/atomic/proc-29167.hdf5', 'data/ppi/processed/atomic/proc-29168.hdf5', 'data/ppi/processed/atomic/proc-29172.hdf5', 'data/ppi/processed/atomic/proc-29164.hdf5', 'data/ppi/processed/atomic/proc-29165.hdf5', 'data/ppi/processed/atomic/proc-29173.hdf5', 'data/ppi/processed/atomic/proc-29169.hdf5'] dataset                 : 100%|██████████| 10/10 [00:00<00:00, 712.76it/s, entry_name=proc-29169.hdf5]


,id,atom_charge,atom_type_0,atom_type_1,atom_type_2,atom_type_3,atom_type_4,atom_type_5,hb_acceptors,hb_donors,...,res_type_16,res_type_17,res_type_18,res_type_19,covalent,distance,electrostatic,same_chain,same_res,vanderwaals
0,atom-ppi:M-P:BA-102765,"[-0.47, 0.235, 0.0, 0.0, 0.265, -0.7, 0.0, 0.0...","[0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.8164346946697534, 5.361942465189271, 4.4459...","[0.0, -0.0, -0.0, -8.694095493121596, 24.84286...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, -0.059433897511631986, -0.14091008980940..."
1,atom-ppi:M-P:BA-103581,"[-0.57, 0.2, -0.0, 0.235, -0.47, 0.5, -0.5, -0...","[0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 5, 5, 1, 0, 0, 0, 0, 0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.4393088619194976, 2.4574232439691777, 3.798...","[0.0, 0.0, -11.711210136338362, 16.89084086674...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, -0.014953759999803005, -0.055365838..."
2,atom-ppi:M-P:BA-104083,"[-0.47, 0.235, 0.0, 0.265, -0.7, 0.0, 0.18, 0....","[0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, ...","[0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 3, 3, 3, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.8161910692435415, 4.665441565382638, 4.8849...","[0.0, -0.0, -8.46649253192547, 23.933588367284...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, -0.11722314475174639, -0.093150415992087..."
3,atom-ppi:M-P:BA-119180,"[-0.47, 0.235, 0.0, 0.0, 0.265, -0.7, -0.7, 0....","[0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 2, ...","[0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 

In [30]:
dataset.save_hist(
    features = "atom_charge",
    fname = os.path.join(processed_data_path, "atomic", "atom_charge"))

Note that some of the features are different from the ones generated with the residue-level queries. There are indeed features in `deeprankcore.features.components` module which are generated only in atomic graphs, i.e. `atom_type`, `atom_charge`, and `pdb_occupancy`, because they don't make sense only in the atomic graphs' representation.